In [20]:
import dask.distributed
import dask.utils
import numpy as np
import planetary_computer as pc
import xarray as xr
import pandas as pd
import re

from IPython.display import display
from pystac_client import Client

from shapely.geometry import Polygon
from odc.stac import configure_rio, stac_load


In [32]:
def get_geometry_from_code(csv_file, experiment_code):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Find the row with the matching experiment code
    row = df[df['Experiment_Code'] == experiment_code]

    if row.empty:
        return None

    # Extract the polygon string
    polygon_str = row.iloc[0]['Field_Polygon']

    # Regular expression to find all pairs of numbers in the string
    coords_str = re.findall(r"\(([^)]+)\)", polygon_str)[0]

    # Split each pair by comma and strip spaces and parentheses
    coords_pairs = coords_str.split(', ')

    # Convert each string pair to a tuple of floats
    area_of_interest = [tuple(map(float, coord.strip('()').split())) for coord in coords_pairs]

    return area_of_interest


In [33]:
# Example usage
csv_file = '/home/nguyen/GitRepos/eo_playground/data/g2f_2017_field_polygon.csv'
csv_df = pd.read_csv(csv_file)
print("List of all experiment codes: \n \n",
    csv_df['Experiment_Code'].values)


List of all experiment codes: 
 
 ['ARH1' 'ARH2' 'COH1' 'DEH1' 'GAH1' 'GAH2' 'IAH1' 'IAH2' 'IAH3' 'IAH4'
 'MIH1' 'MNH1' 'MOH1' 'NCH1' 'NEH1' 'NEH2' 'NEH3' 'NEH4' 'NYH1' 'NYH2'
 'NYH3' 'OHH1' 'ONH1' 'ONH2' 'SCH1' 'TXH1' 'WIH1' 'WIH2']


In [36]:
experiment_code = 'NEH1'
area_of_interest = get_geometry_from_code(csv_file, experiment_code)
print(area_of_interest)


[(-96.409, 41.162), (-96.409, 41.162), (-96.409, 41.162), (-96.409, 41.162), (-96.409, 41.162)]


In [42]:
client = dask.distributed.Client(dashboard_address='localhost:8787')
configure_rio(cloud_defaults=True, client=client)

display(client)
plancomp_catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")


/home/nguyen/miniconda3/envs/geoenv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34053 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34053/status,
Dashboard: http://127.0.0.1:34053/status,Workers: 4
Total threads: 16,Total memory: 31.02 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43073,Workers: 4
Dashboard: http://127.0.0.1:34053/status,Total threads: 16
Started: Just now,Total memory: 31.02 GiB
Comm: tcp://127.0.0.1:41551,Total threads: 4
Dashboard: http://127.0.0.1:42605/status,Memory: 7.75 GiB
Nanny: tcp://127.0.0.1:43761,


In [43]:
# Create a polygon from the area of interest
geometry = Polygon(area_of_interest)

# Define the time period
start_date = '2022-01-01'
end_date = '2022-01-31'

# Search for data in the Sentinel-2 collection that intersects with the geometry
# during the specified time period
search = plancomp_catalog.search(
    collections=['sentinel-2-l2a'],
    datetime=f"{start_date}/{end_date}",
    intersects=geometry,
)


In [44]:
# Fetch the items
items = list(search.items())
print(f"Found {len(items)} items.")

# Extract the STAC items and return them
s2_items = [item.to_dict() for item in items]


Found 23 items.
